# How to set up ROMS/CROCO simulation

Here we will see how to set up ROMS/CROCO files and time for Pyticles. 

Method is based using Modules/R_files.py Classes files and load.

___
## Setting up files

To set up your files you must chose a simulation name and add a switch in 
files Class where you define all files parameters. Here is an example of a
croco simulation named `polgyr`

```Python
# in R_files.py in files()
elif 'polgyr' in simul:
    
    '''
    time 00000 is 2001-01-10 14:53:20
    time 06511 is 2009-12-10 02:53:20
    
    conversion to uncompressed for 2006 = from 3620 to ...
    '''
    
    self.realyear = True
    self.realyear_origin = datetime(1999,1,1)
    self.model = 'croco'
    self.digits = 5

    folder= '/home/datawork-lops-osi/mlecorre/POLGYR/HIS/'
    self.grd='/home/datawork-lops-osi/mlecorre/POLGYR/INIT/polgyr_grd.nc'

    self.his=folder +'polgyr_his.'

    self.frc=folder + '/polgyr_his.03360.nc'
    self.wind=folder + '/polgyr_his.03360.nc'
    self.tfile=20
    self.tstart=0
    self.tend=1000

```
    
Configuration name: 'polgyr'

### model

model used to produce file. It is crucial to determine file names and
and to update file time stepping

- **croco/ucla:** files like `'polgyr_his_03660.nc'`. Note that number of digits can
    be defined by user

- **croco_clim:** CROCO naming for climatology runs with fixed number of
  outputs per file (usually 30 per month). `roms_his_Y2000M1.nc`

- **croco_inter:** CROCO naming for interannual runs with varying number of
  days per month. Also `roms_his_Y2000M1.nc` but 31 days in this one.

- **croco_xios** CROCO naming with XIOS CPP key:  `croco_1999-01-25-1999-01-29.nc`

### file naming and location

- **his** is used as a suffix for file naming

- **grid** should point to grid file (or file with grid parameters)

- **frc** and **wind** are optionnal

- **fileformat**: prefix of the file `.nc` most likely

### metadata parameters

- **realyear** is True for real calendar, otherwise 360 days and 30 days per month

- **realyear_origin**: datetime used for t=0 in croco simulation.
  This is also the date used to load at __init__ method of load()
  Class from R_files.py

- **realyear_tstart**: date that matched t_start=0 for Pyticles.
  This is very usefull when spinup is removed on disk to save some space.
  Typicall case is to remove a few years, then origin date and start date
  are different. 

- **digits**: number of digits to name files (2: 00, 3: 000, ...)

- **dtfile**: time step of ROMS/CROCO file in seconds

- **tfile** number of time steps per file (when fixed)

- **tstart**: time step within CROCO/ROMS file to start from. Usually it is
  set to zero, but it is possible to start from any time step between 0 and
  the number of time steps within the first file 

- **tend**: last time of simulation, take it big.

___
## practical case

It is a good practise before running to Pyticles to ensure that files are
correctly named and that time stepping is alright.


In [7]:
import sys
import numpy as np

# pyticles related modules and tools
sys.path.append("../../Modules/")
from R_files import load

here we load CROCO files from Polgyr simulation (available on Datarmor)

output logs should give you some information about simulation files and variables.

In [10]:
my_simul = 'polgyr'
simul = load(my_simul)

simulname is polgyr
args (['polgyr'],)
args[0] ['polgyr']
len(args[0]) 1
time of simulation is: 0
coord
loading /home/datawork-lops-osi/mlecorre/POLGYR/INIT/polgyr_grd.nc
loading /home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.00000.nc
get domain [0,10000,0,10000,[1,1000,1]] [0,10
coordmax
loading /home/datawork-lops-osi/mlecorre/POLGYR/INIT/polgyr_grd.nc
loading /home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.00000.nc
get domain [0,1e9,0,1e9,[1,1e9,1]] [0,1e
cst
read Cs_r in ncfile.Cs_r
dt
dt is read in  /home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.00000.nc
[0, 1602, 0, 2002, array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80])]
ncname0,ny1,ny2,nx1,nx2
/home/dataw

To check that time stepping is alright

- time 00000 is 2001-01-10 14:53:20

- and time 06511 is 2009-12-10 02:53:20

Using load method, and filetime, infiletime attributes

In [20]:
simul.update(time=6511)

# currently loaded date
print(f"{simul.date=}")

# current year
print(f"{simul.year=}")

# time step in file
print(f"{simul.infiletime=}")

time of simulation is: 6511
/home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.06500.nc
simul.date='Dec 10 - 02:53'
simul.year=2009
simul.infiletime=11
